<h1><center>ESADE Datahub Workshop</center></h1>
<h2><center>House Price Prediction</center></h2>

The aim of this project is to build a Machine Learning model in order to predict the appropriate price of a house given a set of features.
We decided to divide our analysis into 5 parts:
 - First look at the problem and general understanding of the variables;
 - Study the main variable ("SalePrice"); 
 - Study how the main variable is related to the other feature;
 - Data Preprocessing:
 - Build a model in order to predict SalePrice 
 
***

#### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
%matplotlib inline
from sklearn.model_selection import cross_val_score, train_test_split, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, ElasticNetCV, BayesianRidge, LassoLarsIC
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import math
from sklearn.preprocessing import MinMaxScaler

In [ ]:
result = pd.read_csv("/content/train.csv")
result

In [ ]:
numeric = result.select_dtypes(include=np.number)
numeric.shape

In [ ]:
result.head()

In [ ]:
result.tail()

In [ ]:
result.info()

In [ ]:
result.describe()

# Our initial considerations 
Looking forward to our columns, we found some variables which can have an high correlation with our main variable SalePrice:
- __Year Built__
- __TotalBsmtSF__
- __GrLivArea__
- __PoolArea__

These are variables related to the conditions of the building, its age and some "extra luxury" features such as __PoolArea__. 
In principle they are all characteristics which can rise the price of an house. 
Another theory we suggested was to consider mainly the "inner" part of the house, such as __KitchenQual__ or __CentralAir__, but these could be too general features which mainly all the houses can have.

Now, with these prior hypotesis, let's dive into the "__SalePrice__" analysis.

# SalePrice Analysis

In [ ]:
y = result['SalePrice']

In [ ]:
y.describe()

In [ ]:
sns.distplot(result['SalePrice']);
print("Skewness coeff. is: %f" % result['SalePrice'].skew())
print("Kurtosis coeff. is: %f" % result['SalePrice'].kurt())

**These** measures of symmetry are useful in order to understand the symmetry of the distribution of our main variable.
Our distribution is highly skewed and present a longer tail on the right. 
The high value of kurtosis can determine an higher probability of outliers values.

# The other variables

In [ ]:
data_year_trend = pd.concat([result['SalePrice'], result['YearBuilt']], axis=1)
data_year_trend.plot.scatter(x='YearBuilt', y='SalePrice', ylim=(0,800000));

In [ ]:
data_bsmt_trend = pd.concat([result['SalePrice'], result['TotalBsmtSF']], axis=1)
data_bsmt_trend.plot.scatter(x='TotalBsmtSF', y='SalePrice', ylim=(0,800000));

In [ ]:
data_GrLivArea_trend = pd.concat([result['SalePrice'], result['GrLivArea']], axis=1)
data_GrLivArea_trend.plot.scatter(x='GrLivArea', y='SalePrice', ylim=(0,800000));

In [ ]:
data_PoolArea_trend = pd.concat([result['SalePrice'], result['PoolArea']], axis=1)
data_PoolArea_trend.plot.scatter(x='PoolArea', y='SalePrice', ylim=(0,800000));

In [ ]:
data = pd.concat([result['SalePrice'], result['OverallQual']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='OverallQual', y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);

By these analysis we discovered that our previsions were quite correct.

__Year Built__ seems to have a slight relation with our main variable, and people, as we thought, tend to buy newer houses. 

 __TotalBsmtSF__ and __GrLivArea__ there seems be a stronger relation with __SalePrice__. 

# Heatmap Correlation Matrix

In [ ]:
corr_matrix = result.corr()
f, ax1 = plt.subplots(figsize=(12,9)) 
ax1=sns.heatmap(corr_matrix,vmax = 0.9); 

Using this kind of plot we can deduce if there's some collinearity between 2 or more variables.
In particoular, there are some white blocks which have to be analyzed:
1. __GarageYrBlt__ and __YearBuilt__
2. __TotRmsAbvGrd__ and __GrLivArea__
3. __TotalBsmtSF__ and __X1stFlrSF__
4. __GarageArea__ and __GarageCars__
 
Knowing the meaning of these pairs of variables seems trivial to notice a collinearity between pairs "1", "3" and "4".
For the "2" pair the difference is slightly more subtle because the house area and the total number of rooms, not always are related. 
For example two houses with the same living area can be inhabited by different number of peoples and so the actual disposition/number of the rooms can be different.

Let's restrict our matrix a bit more.

In [ ]:
corrmat = result.corr()
top_corr_features = corrmat.index[abs(corrmat["SalePrice"])>0.5]
plt.figure(figsize=(9,9))
g = sns.heatmap(result[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
var = result[result.columns[1:]].corr()['SalePrice'][:]
var

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(result[cols], height = 2.5)
plt.show();

# Dealing with Null Values

Now our goal is to deal with null values and try to understand for each one what can we do: drop or fill them? 

In [ ]:
total_null = result.isnull().sum().sort_values(ascending=False) #First sum and order all null values for each variable
percentage = (result.isnull().sum()/result.isnull().count()).sort_values(ascending=False) #Get the percentage
missing_data = pd.concat([total_null, percentage], axis=1, keys=['Total', 'Percentage'])
missing_data.head(20)

We have to do some considerations. 
Let's divide our null values into 2 groups:
 - __PoolQC__, __MiscFeature__, __Alley__, __Fence__, __FireplaceQu__ and __LotFrontage__.
These are all variables which presents many null values. In general, by common opinion, we can discourage variables which have more than 15% of missing values. 
These are not vital information for someone who wants to buy an house, such as __FireplaceQu__ or, for example, many houses doesn't have an __Alley__ access. We can drop them.

The second group:
 - __GarageX__ properties
If we look carefully, all of these variables have the same number of null values! Maybe this can be a strange coincidence, or just that they all refer to the same variable Garage, in which "Na" means "There is no Garage". The same occurs for __BsmtX__ and MasVnr__, which means that we will have to deal with them afterwards.

In [ ]:
result = result.drop((missing_data[missing_data["Percentage"] > 0.15]).index,1) 
result.isnull().sum()

# Categorical vs Numerical Variables





### Handling Correlation and Variance

In the following block we are going to remove some columns.
Let's see why

- **Zero Variance Problem**: If the variance is low or close to zero, then a feature is approximately constant and will not improve the performance of the model. In that case, it should be removed.
- **Multicollinearity**: It is the occurrence of high intercorrelations among two or more independent variables in a multiple regression model. Why avoid that? 
- **Other variables are composed by sub-features**

In [ ]:
# What I mean by Zero Variance Variables
result['MiscVal'].value_counts()

In [ ]:
#Zero variance
del result["KitchenAbvGr"]
del result["YrSold"]
del result["MoSold"]
del result["MiscVal"]
del result["ScreenPorch"]
del result["X3SsnPorch"]
del result["BsmtHalfBath"]
del result["LowQualFinSF"]
del result["OverallCond"]
del result["EnclosedPorch"]
del result["MSSubClass"]
del result["X1stFlrSF"]
del result["YearBuilt"]
del result["YearRemodAdd"] 
del result["BsmtFinSF2"] 
del result["PoolArea"] 
del result["GarageYrBlt"] 
del result["GarageCond"] 
del result["Street"]
del result["LandContour"]
del result["Utilities"]
del result["LandSlope"]
del result["Condition2"]
del result["RoofMatl"]
del result["BsmtFinType2"] 
del result["Electrical"] 
del result["Condition1"]
del result["BldgType"]
del result["HouseStyle"]
del result["Exterior1st"]
del result["Exterior2nd"]
del result["Foundation"]
del result["CentralAir"]
del result["Functional"]
del result["SaleType"]
del result["SaleCondition"]
del result["RoofStyle"]
del result['ExterCond']
del result['BsmtCond']

#Multicollinearity
del result["GarageArea"] 
del result["TotRmsAbvGrd"] 

#Variable composition
del result["BsmtFinSF1"] #Because BsmtFinSF1 + BsmtUnfSF + BsmtFinSF2 = TotalBsmtSF
del result["BsmtUnfSF"] #Because BsmtFinSF1 + BsmtUnfSF + BsmtFinSF2 = TotalBsmtSF

### Encoding of Rank Categorical Features

In [ ]:
result['ExterQual'].value_counts()

In [ ]:
#Here we encode ExterQual in a rank
result.loc[result['ExterQual'] == "Ex", 'ExterQual'] = 5
result.loc[result['ExterQual'] == "Gd", 'ExterQual'] = 4
result.loc[result['ExterQual'] == "TA", 'ExterQual'] = 3
result.loc[result['ExterQual'] == "Fa", 'ExterQual'] = 2
result.loc[result['ExterQual'] == "Po", 'ExterQual'] = 1
result['ExterQual']

In [ ]:
#Here we encode HeatingQC in Rank
result.loc[result['HeatingQC'] == "Ex", 'HeatingQC'] = 5
result.loc[result['HeatingQC'] == "Gd", 'HeatingQC'] = 4
result.loc[result['HeatingQC'] == "TA", 'HeatingQC'] = 3
result.loc[result['HeatingQC'] == "Fa", 'HeatingQC'] = 2
result.loc[result['HeatingQC'] == "Po", 'HeatingQC'] = 1
result['HeatingQC']

In [ ]:
#Here we encode BsmtFinType1 in Rank
result.loc[result['BsmtFinType1'] == "GLQ", 'BsmtFinType1'] = 6
result.loc[result['BsmtFinType1'] == "ALQ", 'BsmtFinType1'] = 5
result.loc[result['BsmtFinType1'] == "BLQ", 'BsmtFinType1'] = 4
result.loc[result['BsmtFinType1'] == "Rec", 'BsmtFinType1'] = 3
result.loc[result['BsmtFinType1'] == "LwQ", 'BsmtFinType1'] = 2
result.loc[result['BsmtFinType1'] == "Unf", 'BsmtFinType1'] = 1
result['BsmtFinType1'].fillna(0, inplace= True)
result['BsmtFinType1']

In [ ]:
#Here we encode BsmtQual in Rank
result.loc[result['BsmtQual'] == "Ex", 'BsmtQual'] = 5
result.loc[result['BsmtQual'] == "Gd", 'BsmtQual'] = 4
result.loc[result['BsmtQual'] == "TA", 'BsmtQual'] = 3
result.loc[result['BsmtQual'] == "Fa", 'BsmtQual'] = 2
result.loc[result['BsmtQual'] == "Po", 'BsmtQual'] = 1
result['BsmtQual'].fillna(0, inplace= True)
result['BsmtQual']

In [ ]:
#Here we encode KitchenQual in Rank
result.loc[result['KitchenQual'] == "Ex", 'KitchenQual'] = 4
result.loc[result['KitchenQual'] == "Gd", 'KitchenQual'] = 3
result.loc[result['KitchenQual'] == "TA", 'KitchenQual'] = 2
result.loc[result['KitchenQual'] == "Fa", 'KitchenQual'] = 1
result['KitchenQual']

In [ ]:
#Here we encode BsmtExposure in Rank
result.loc[result['BsmtExposure'] == "Gd", 'BsmtExposure'] = 4
result.loc[result['BsmtExposure'] == "Av", 'BsmtExposure'] = 3
result.loc[result['BsmtExposure'] == "Mn", 'BsmtExposure'] = 2
result.loc[result['BsmtExposure'] == "No", 'BsmtExposure'] = 1
result['BsmtExposure'].fillna(0, inplace= True)
result['BsmtExposure']

In [ ]:
#Here we encode GarageQual in Rank
result.loc[result['GarageQual'] == "Ex", 'GarageQual'] = 5
result.loc[result['GarageQual'] == "Gd", 'GarageQual'] = 4
result.loc[result['GarageQual'] == "TA", 'GarageQual'] = 3
result.loc[result['GarageQual'] == "Fa", 'GarageQual'] = 2
result.loc[result['GarageQual'] == "Po", 'GarageQual'] = 1
result['GarageQual'].fillna(0, inplace= True)
result['GarageQual']

In [ ]:
del result["GarageQual"] #Large majority of values are 3

In [ ]:
#Here we encode GarageFinish in Rank
result.loc[result['GarageFinish'] == "Fin", 'GarageFinish'] = 4
result.loc[result['GarageFinish'] == "RFn", 'GarageFinish'] = 3
result.loc[result['GarageFinish'] == "Unf", 'GarageFinish'] = 2
result['GarageFinish'].fillna(0, inplace= True)
result['GarageFinish']

In [ ]:
#HERE WE FILL THE LAST NAs IN THOSE VARIABLES WHICH WE CAN NOT RANK
result['MasVnrType'].fillna("None", inplace= True)
result['MasVnrArea'].fillna(0, inplace= True)
result['GarageType'].fillna("No Garage", inplace= True)

# Outliers Detection

In [ ]:
#Take only numerical variables for outliers detection
numeric = result.select_dtypes(include=np.number) 
n_features = numeric.columns
n_features

### IQR analysis
The Interquartile range formula measures the variability, based on dividing an ordered set of data into quartiles.

In [ ]:
Q1 = result[n_features].quantile(0.25)
Q3 = result[n_features].quantile(0.75)
IQR = Q3 - Q1

result = result[~((result[n_features] < (Q1 - 1.5 * IQR)) |(result[n_features] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
# Difference between previous columns and new numerical
result = result[result.columns.difference(numeric.columns)]

### Normalization

Do we actually need that? 
Normalization is a good technique to use when we do not know the distribution of your data or when you know the distribution is not bell-shaped (Gaussian)

In [ ]:
#numeric = (numeric-numeric.mean())/numeric.std() 

In [ ]:
#Concatenate Result + Numeric and reset indexes
result = pd.concat([result, numeric], axis=1)
result = result.reset_index()

In [ ]:
result

In [ ]:
#Adjust result df removing some NaN's rows obtained from concatenation and the index columnn.
result = result[result["BsmtExposure"].notna()]
del result['index']

### Dummy Variables

In [ ]:
data_train = pd.get_dummies(result)
data_train

In [ ]:
data_train = data_train.fillna(0)

In [ ]:
y = data_train['SalePrice']
data = data_train.drop(['SalePrice'], axis=1)
xtrain, xvalid, ytrain, yvalid = train_test_split(data, y, test_size=0.3, random_state=40)

# Model

### Random Forest

In [ ]:
regressor = RandomForestRegressor(n_estimators=300, random_state=0)
regressor.fit(xtrain,ytrain)

In [ ]:
predicted = regressor.predict(xvalid)
predicted

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(yvalid, predicted))
print('MSE:', metrics.mean_squared_error(yvalid, predicted))
print('RMSE:', np.sqrt(metrics.mean_squared_error(yvalid, predicted)))

### Gradient Boosting

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(xtrain, ytrain)
rmse = math.sqrt(mean_squared_error(yvalid, GBoost.predict(xvalid)))

In [ ]:
predicted = GBoost.predict(xvalid)

In [ ]:
print('MAE:', metrics.mean_absolute_error(yvalid, predicted))
print("RMSE: %.4f" % rmse)

# Conclusion and further improvements

How can we improve our research ?